<a href="https://colab.research.google.com/github/rihemmaarefe/TestingObjSubj/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -q streamlit

In [ ]:
# !pip install altair

In [ ]:
# !apt-get upgrade sqlite3

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd 
import sqlite3
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.naive_bayes import MultinomialNB 
import matplotlib.pyplot as plt
import altair as alt


# Load the data 
data = pd.read_csv('/content/drive/MyDrive/stage/trainData.csv') 
X = data['meaning'] 
y = data['category']

# Preprocess the data using the CountVectorizer 
vectorizer = CountVectorizer(stop_words='english') 
X_counts = vectorizer.fit_transform(X) 

# Train a machine learning model 
model = MultinomialNB() 
model.fit(X_counts, y) 

# Define a function to predict the category of new text 
def predict_category(text):
    text_counts = vectorizer.transform([text])
    prediction = model.predict(text_counts)
    probabilities = model.predict_proba(text_counts)[0]
    return prediction[0],probabilities

# Define a function to plot the probabilities of each category 
def plot_probabilities(text):
    text_counts = vectorizer.transform([text])
    probabilities = model.predict_proba(text_counts)[0]
    categories = model.classes_
    fig, ax = plt.subplots(figsize=(6, 2))
    ax.bar(df_proba['label'], df_proba['probability'])
    ax.set_xlabel('Category')
    ax.set_ylabel('Probability')
    ax.set_title('Category Probabilities')
    for i, prob in enumerate(df_proba['probability']):
      ax.text(i, prob + 0.05, f"{prob*100:.2f}%", ha='center')
    st.pyplot(fig)



# Define the SessionState class to store the dataframe
class SessionState:
    def __init__(self):
        self.results_df = pd.DataFrame(columns=['text_input', 'predicted_label'])

# Create the Streamlit app and initialize the SessionState object
st.title('Objective vs. Subjective Text Classifier')
state = SessionState()

# Connect to the database
conn = sqlite3.connect('results.db')
c = conn.cursor()

# Create a table for the results if it doesn't exist
c.execute('''CREATE TABLE IF NOT EXISTS results
             (text_input TEXT, predicted_label TEXT)''')

 # Add a text input and a submit button
text_input = st.text_input('Enter some text:')
if st.button('Submit'):
    # Predict the category and store the input and output data in the dataframe
    prediction, probabilities = predict_category(text_input)
    state.results_df = state.results_df.append({'text_input': text_input, 'predicted_label': prediction}, ignore_index=True)
    
    # Insert the results into the database
    c.execute("INSERT INTO results VALUES (?, ?)", (text_input, prediction))
    conn.commit()
    
    # Plot the probabilities of each category
    # plot_probabilities(text_input)

    st.success("Data Submitted")
    
    res_col1 ,res_col2 = st.columns(2)
    with res_col1:
      st.info("Prediction")
      st.write(prediction)

    with res_col2:
      st.info("Probability")
      #  plot
      categories, probabilities = zip(*sorted(zip(model.classes_, probabilities), key=lambda x: x[1], reverse=True))
      df_proba = pd.DataFrame({'label': categories, 'probability': probabilities})

      # visz
      fig = alt.Chart(df_proba).mark_bar().encode(x='label',y='probability')
      st.altair_chart(fig,use_container_width=True)	


# Fetch the results from the database and display them in a table
c.execute("SELECT * FROM results")
results = c.fetchall()
results_df = pd.DataFrame(results, columns=['text_input', 'predicted_label'])
st.write(results_df)
st.write('made by Rihem Maaref')

# Close the database connection
conn.close()

    

Overwriting app.py


In [ ]:
# !npm install localtunnel

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 4.562s
your url is: https://purple-roses-win-35-194-90-232.loca.lt
